In [1]:
import hopsworks
import pandas as pd
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os
from sklearn.ensemble import RandomForestRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194714
Connected. Call `.close()` to terminate connection gracefully.


#### Get or Create a new feature view

In [3]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
flight_fg = fs.get_feature_group(name="flight_data_v2", version=1)
query = flight_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="flight_data_v2",
                                  version=1,
                                  description="Read from Flight Delay dataset",
                                  labels=["dep_delay_new"],
                                  query=query)

#### Create a new train test split

In [8]:
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(test_size=0.2)

Finished: Reading data from Hopsworks, using ArrowFlight (17.63s) 


#### Read from existing train test split

In [3]:
#flight_fg = fs.get_feature_group(name="flight_data", version=1)
#query = flight_fg.select_all()
feature_view = fs.get_feature_view(name="flight_data_v2",version=1)
# get a training dataset with train and test splits
X_train, X_test, y_train, y_test = feature_view.get_train_test_split(training_dataset_version=8)

Finished: Reading data from Hopsworks, using ArrowFlight (17.01s) 


In [14]:
X_train

,year,month,day_of_month,day_of_week,origin_airport_id,origin_wac,dest_airport_id,dest_wac,crs_dep_time,crs_arr_time,hourlydewpointtemperature,hourlydrybulbtemperature,hourlyprecipitation,hourlyrelativehumidity,hourlystationpressure,hourlyvisibility,hourlywinddirection,hourlywindgustspeed,hourlywindspeed
0,2022,6,29,3,13303,33,13930,41,1330,1602,21.7,28.3,0.0,67.0,1019.30,16.00,250,0.0,6.0
1,2021,11,16,2,14100,23,11697,33,800,1048,-2.2,4.4,0.0,63.0,1022.01,16.09,250,0.0,4.0
2,2022,9,22,4,14100,23,12266,74,1903,2150,8.3,20.6,0.0,45.0,1004.06,16.00,320,0.0,11.0
3,2022,2,9,3,13930,41,11292,82,1245,1431,-1.7,4.4,0.0,65.0,981.04,16.00,260,0.0,8.0
5,2021,8,2,1,13303,33,12892,91,1930,2145,25.0,30.0,0.0,75.0,1016.26,16.00,150,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662950,2021,6,7,1,10397,34,12892,91,1514,1649,21.1,27.2,0.0,69.0,981.38,16.09,200,0.0,4.0
1662951,2022,11,12,6,13204,33,11057,36,600,748,20.0,20.0,0.0,100.0,1010.84,0.40,0,0.0,0.0
1662952,2022,1,11,2,10397,34,13204,33,715,847,-8.9,-0.6,0.0,54.0,992.89,16.00,320,0.0,3.0
1662953,2021,5,22,6,11618,21,12889,85,2155,30,14.4,27.8,0.0,44.0,1015.58,16.09,280,0.0,4.0


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1330364 entries, 1 to 1662955
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   year                       1330364 non-null  int64  
 1   month                      1330364 non-null  int64  
 2   day_of_month               1330364 non-null  int64  
 3   day_of_week                1330364 non-null  int64  
 4   origin_airport_id          1330364 non-null  int64  
 5   origin_wac                 1330364 non-null  int64  
 6   dest_airport_id            1330364 non-null  int64  
 7   dest_wac                   1330364 non-null  int64  
 8   crs_dep_time               1330364 non-null  int64  
 9   crs_arr_time               1330364 non-null  int64  
 10  hourlydewpointtemperature  1330364 non-null  float64
 11  hourlydrybulbtemperature   1330364 non-null  float64
 12  hourlyprecipitation        1330364 non-null  float64
 13  hourlyrelativehum

In [11]:
y_train

,dep_delay_new
1,0.0
2,0.0
3,0.0
5,68.0
6,0.0
...,...
1662949,26.0
1662950,2.0
1662953,4.0
1662954,14.0


In [12]:
y_train.values.ravel()

array([ 0.,  0.,  0., ...,  4., 14.,  0.])

#### Random Forest Regressor Model

In [13]:
# Initialize the Random Forest classifier
model_RF = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF.fit(X_train, y_train)
# Predict labels for the test set
y_pred = model_RF.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 47.8940051408346
MAE: 20.04161412364382
R-squared: 0.04646489988019231
Mean Squared Error (MSE): 2293.835728430291


In [14]:
y_train_np=y_train.values.ravel()
X_train_np=X_train.values
X_test_np = X_test.values
y_test_np=y_test.values.ravel()

In [ ]:
# Initialize the Random Forest classifier
model_RF_np = RandomForestRegressor(max_depth=10,random_state=1)  # You can adjust parameters here

# Train the classifier
model_RF_np.fit(X_train_np, y_train_np)
# Predict labels for the test set
y_pred_np = model_RF_np.predict(X_test_np)

mse = mean_squared_error(y_test_np, y_pred_np)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test_np, y_pred_np)

# Calculate R-squared
r2 = r2_score(y_test_np, y_pred_np)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

#### Linear Regression

In [16]:
# Initialize the Linear Regression model
model_lin_reg = LinearRegression()

# Fit the model on the training data
model_lin_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model_lin_reg.predict(X_test)

# Calculate the Mean Squared Error (MSE) on test set

mse = mean_squared_error(y_test, y_pred)
# Calculate RMSE
rmse = np.sqrt(mse)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R-squared: {r2}")
print(f"Mean Squared Error (MSE): {mse}")

RMSE: 48.61102110761538
MAE: 20.818163650160685
R-squared: 0.017700644805569876
Mean Squared Error (MSE): 2363.031373125028


#### Neural Network

In [17]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [18]:
# Normalize features (optional but often recommended for neural networks)
# outputs np arrays
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_test_scaled = scaler.transform(X_test_tensor)

In [19]:
# Convert the scaled NumPy arrays back to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

In [20]:
# Convert to PyTorch Dataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [23]:
# Initialize the model
input_size = X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Train the model
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    #convert to rmse
    train_loss = running_loss / len(train_loader)
    train_rmse = torch.sqrt(torch.tensor(train_loss))
    
    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, targets).item()
        val_loss /= len(val_loader)

    val_rmse = torch.sqrt(torch.tensor(val_loss))

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss MSE: {train_loss}, Train Loss RMSE: {train_rmse}, Val Loss MSE: {val_loss}, Val RMSE: {val_rmse}")

Epoch 1/25, Train Loss MSE: 2407.4271894304384, Train Loss RMSE: 49.0655403137207, Val Loss MSE: 2323.6752567610556, Val RMSE: 48.20451736450195
Epoch 2/25, Train Loss MSE: 2389.8179003234295, Train Loss RMSE: 48.885765075683594, Val Loss MSE: 2314.1992039749293, Val RMSE: 48.10612487792969
Epoch 3/25, Train Loss MSE: 2382.5500641100366, Train Loss RMSE: 48.81137466430664, Val Loss MSE: 2311.9186152615453, Val RMSE: 48.08241653442383
Epoch 4/25, Train Loss MSE: 2379.301379003904, Train Loss RMSE: 48.77808380126953, Val Loss MSE: 2303.4545022958237, Val RMSE: 47.99431610107422
Epoch 5/25, Train Loss MSE: 2374.979973282908, Train Loss RMSE: 48.733768463134766, Val Loss MSE: 2303.423386651414, Val RMSE: 47.99399185180664
Epoch 6/25, Train Loss MSE: 2372.982733050942, Train Loss RMSE: 48.71327209472656, Val Loss MSE: 2304.653378919521, Val RMSE: 48.006805419921875
Epoch 7/25, Train Loss MSE: 2371.576577881963, Train Loss RMSE: 48.69883728027344, Val Loss MSE: 2302.475309358369, Val RMSE: 4

#### Save to Hopsworks Model Registry 

In [24]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'iris_model' directory will be saved to the model registry. Create the dir, first.
model_dir="flight delay model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/flight_delay_model.pkl")
#fig.savefig(model_dir + "/confusion_matrix.png")    

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
#input_schema = Schema(X_train_tensor)
#output_schema = Schema(y_train_tensor)
#model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
fligt_delay_model = mr.python.create_model(
    name="flight_delay_model", 
    #metrics={"accuracy" : metrics['accuracy']},
    metrics={"val MSE" : 2252},
    #model_schema=model_schema,
    description="Flight Delay Predictor no schema"
)

# Upload the model to the model registry, including all files in 'model_dir'
fligt_delay_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading: 100.000%|██████████| 46789/46789 elapsed<00:01 remaining<00:001,  3.23it/s]
Model export complete: 100%|██████████| 6/6 [00:08<00:00,  1.36s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/194714/models/flight_delay_model/1


Model(name: 'flight_delay_model', version: 1)

### Example showing how to load the NN model from Model Registry and how to use it for inference

Class definition needed due to how python loads saved pytorch models. The class definition is used to load the pickle model. If the pytorch model was saved as a state dict it could also be loaded using state_dict without defining class definition

In [26]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

mr = project.get_model_registry()
model = mr.get_model("flight_delay_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/flight_delay_model.pkl")


Connected. Call `.close()` to terminate connection gracefully.


In [37]:
#Data sample used for inference
i_test=0
a=X_train.iloc[i_test]
a=torch.tensor(a, dtype=torch.float32)

#Fit scaler to how it was fitted during training
scaler = StandardScaler()
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_train_scaled = scaler.fit_transform(X_train_tensor)
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)

#transform data sample
a=scaler.transform(a.reshape(1, -1))
print(a)
print(X_train_tensor[i_test])

# transform np array to torch tensor
a_tensor=torch.tensor(a, dtype=torch.float32)

print(a_tensor)
print(model(a_tensor))
print(y_train.iloc[i_test])

[[-1.05282281  1.24782137  0.02323961 -0.99690901  1.18383447 -1.1339801
  -0.67183469 -0.77138126 -1.05325297 -0.80630688 -1.06317687 -1.3894594
  -0.09987404  0.10331179  0.70543781  0.33138419  0.66613952 -0.35530529
  -0.04343485]]
tensor([-1.0528,  1.2478,  0.0232, -0.9969,  1.1838, -1.1340, -0.6718, -0.7714,
        -1.0533, -0.8063, -1.0632, -1.3895, -0.0999,  0.1033,  0.7054,  0.3314,
         0.6661, -0.3553, -0.0434])
tensor([[-1.0528,  1.2478,  0.0232, -0.9969,  1.1838, -1.1340, -0.6718, -0.7714,
         -1.0533, -0.8063, -1.0632, -1.3895, -0.0999,  0.1033,  0.7054,  0.3314,
          0.6661, -0.3553, -0.0434]])
tensor([[7.3723]], grad_fn=<AddmmBackward0>)
dep_delay_new    0.0
Name: 1, dtype: float64
